In [1]:
print("all_ok")

all_ok


In [2]:
from langchain_groq import ChatGroq

c:\LLMOPS\Project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
model=ChatGroq(model="qwen/qwen3-32b")

In [7]:
model.invoke("What is the capital of France?").content

"<think>\nOkay, so the question is asking for the capital of France. Let me think. I remember that France is a country in Europe. Capitals are usually the main cities where the government is located. I think the capital of France is Paris. Wait, but I should be sure. Sometimes people mix up capitals with other major cities. For example, Lyon is a big city in France, but is that the capital? No, I'm pretty sure Paris is the capital because that's where the Eiffel Tower is, and it's a well-known city. Also, the French government is based there. Let me double-check in my mind. Other countries with similar names: Spain's capital is Madrid, Italy's is Rome, Germany's is Berlin. France's should be Paris. Yeah, I think that's right. I don't recall any other city being the capital. Maybe Marseille? No, that's a port city. Definitely Paris.\n</think>\n\nThe capital of France is **Paris**. It is not only the political and administrative center of the country but also a major cultural and economi

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode("what is tree")


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 399.15it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


1.Data ingestion

In [10]:
from langchain.document_loaders import PyPDFLoader

ModuleNotFoundError: No module named 'langchain.document_loaders'